PREREQUISITES:
* You must have dbt installed.
* The working/client dbt deployment must be using the `dbt-codegen` package, see install instructions here: https://github.com/dbt-labs/dbt-codegen

You'll need to specify the following inputs for this notebook:
* `WORKING_FOLDER_WITH_DBT_ARC_FUNCTION_MODELS`: ABSOLUTE PATH. this is probably a client folder. It contains models that invoke the dbt_arc_functions macros that you'd like to document. For instance, if I wanted to document the `stitch_sfmc_email`'s `staging` macros, I'd find a client that uses that connector, find the folder that uses those macros, and enter its absolute path here. 
* `DBT_ARC_FUNCTIONS_MACRO_DOCUMENTATION_FOLDER`: ABSOLUTE PATH. this is a folder within dbt-arc-functions. It is within the documentation folder and specifies which macro you're trying to document. To take the above example, we're trying to document the `stich_sfmc_email`'s `staging` macros. Therefore your path would look something like `/Users/user/Desktop/dbt-arc-functions/documentation/stitch_sfmc_email/staging`.

In [1]:
WORKING_FOLDER_WITH_DBT_ARC_FUNCTION_MODELS= "REPLACE WITH WORKING CLIENT FOLDER"
DBT_ARC_FUNCTIONS_MACRO_DOCUMENTATION_FOLDER= "REPLACE WITH DBT ARC FUNCTIONS MACRO FOLDER"

In [2]:
import os
import shutil
import ruamel.yaml
import subprocess
from utils import check_dbt_installed, check_profiles_file, run_dbt_subprocess

In [3]:

print("Checking if dbt installed...\n")
check_dbt_installed()
print("Checking if you have a profiles.yml in the .dbt folder...\n")
check_profiles_file()
print("Installing dbt deps...\n")
process = subprocess.run(
        f"cd {WORKING_FOLDER_WITH_DBT_ARC_FUNCTION_MODELS} ;dbt deps",
        capture_output=True,
        shell=True,
        check=False)
print(f"Looping through files in {WORKING_FOLDER_WITH_DBT_ARC_FUNCTION_MODELS}...\n")
for file in os.listdir(WORKING_FOLDER_WITH_DBT_ARC_FUNCTION_MODELS):
    if file.endswith('.sql'):
        print(f"Working on: {file}")
        doc_file = file.replace('.sql','.yml')
        doc_path = os.path.join(DBT_ARC_FUNCTIONS_MACRO_DOCUMENTATION_FOLDER, doc_file)
        if not os.path.exists(doc_path):
            model = file[:-4]
            bash_command = (f"cd {WORKING_FOLDER_WITH_DBT_ARC_FUNCTION_MODELS} ; "
                            f"""dbt run-operation generate_model_yaml --args '{{"model_names": ["{model}"], "upstream_descriptions": true}}'"""
            )
            output = run_dbt_subprocess(bash_command)
            index_of_version = output.find('version')
            cleaned_output = output[index_of_version:]
            with open(doc_path, 'w') as f:
                f.write(cleaned_output)
            print(f"Wrote to {doc_path}!\n")
        else:
            print(f"{doc_path} already exists, will not overwrite\n")

Checking if dbt installed...

Checking if you have a profiles.yml in the .dbt folder...

Installing dbt deps...

Looping through files in /Users/tim/Desktop/arc-mercycorps-dbt/models/staging/email...

Working on: stg_email_complaints_rollup_unioned.sql
Working on: stg_email_actions_rollup_unioned.sql
Working on: stg_email_opens_rollup_unioned.sql
Working on: stg_email_campaign_dates_rollup_unioned.sql
Working on: stg_email_transactions_sourced_rollup_unioned.sql
Working on: stg_email_clicks_rollup_unioned.sql
Working on: stg_email_bounces_rollup_unioned.sql
Working on: stg_email_recipients_rollup_unioned.sql
Working on: stg_email_campaigns_rollup_unioned.sql
Working on: stg_email_unsubscribes_rollup_unioned.sql
Working on: stg_email_jobs_unioned.sql
